## Chemical reaction in a simulation box

Computational techniques are used to study chemical reactions without stepping in an laboratory.
Here we are going to study one of the simplest but most important reactions in clean energy science which is the combination of $O_2$ and $H_2$ to form $H_2O$:

$2H_2 + O_2 \rightarrow 2H_2O$

the first step is to create a simulation box that contains the reagents of interest:

In [ ]:
from ase.build import molecule
from ase.visualize import view

atoms1 = molecule('O2')
atoms2 = molecule('H2')
atoms2.translate([2, 0, 0])

atoms3 = molecule('H2')
atoms3.translate([-2, 0,0])

atoms = atoms1 + atoms2 + atoms3
atoms.center(5)

atoms.set_pbc([1,1,1])

view(atoms)

The reaction will be simulated using a technique known as Molecular Dynamics (MD), which performs a real time simulation of the motion of atoms and molecules.

In [ ]:
from ase.calculators.emt import EMT
from ase.io.trajectory import Trajectory
from ase.md import VelocityVerlet
from ase import units
from ase import Atoms
import ase.io

atoms.calc = EMT()

dyn =  VelocityVerlet(atoms, 1 * units.fs)


traj = Trajectory('traj/h2o.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)

# Now run the dynamics
dyn.run(100)

frames = ase.io.read('traj/h2o.traj', index=':')
view(frames)


### Reaction energy

Understanding if a chemical reaction is energetically favorable (releases energy) or is energetically unfavorable (requries energy) is one of the fundaments of chemistry  known as thermodynamics.

Accurate calculations of reaction energies require computational techniques based on quantum mechanics known as 'ab initio' or first principles calculations.

Here, we will used the most widely used first principle technique known as Density Functional Theory (DFT). Calculations using this approximation can be computationally demanding and we will submit this tasks as jobs at Yale Grace supercomputer in the box below:

In [ ]:
!sbatch qe/O2.sh
!sbatch qe/H2.sh
!sbatch qe/H2O.sh

You can check if the supercomputer has finished the tasks with the command below:

In [ ]:
!squeue --me

In the box below, we will extract the total energy of $H_2$, $O_2$ and $H_2O$ from the DFT calculation output and compute the reaction energy.

In [ ]:
from ase.calculators.espresso import Espresso
from ase.io.espresso import write_espresso_in
import ase
from ase.io import read, write

!sed -i 's/magn=//' qe/O2.out

H2_opt= ase.io.read('qe/H2.out')
H2_energy = H2_opt.get_total_energy()
O2_opt= ase.io.read('qe/O2.out')
O2_energy = O2_opt.get_total_energy()
H2O_opt = ase.io.read('qe/H2O.out')
H2O_energy = H2O_opt.get_total_energy()

ev2kjmol = 96.48
kj2kcal = 0.239
h2_weight = 2.01568

ev_reaction = H2O_energy - ( H2_energy + 0.5*O2_energy)
kj_reaction = ev_reaction * ev2kjmol
kcal_gram = kj_reaction*kj2kcal*(1/h2_weight)

print('Energy %5.2f eV' % ev_reaction)
print('Energy %5.2f kJ/mol' % kj_reaction)
print('Energy %5.2f kcal/g' % kcal_gram)


## Water dimer: Classical model

Geometry optimization is the study of the lowest energy structure that an atomic structure can achieved. It is key in the determination of any property since, in the absence of external perturbations, atoms arrange in the lowest energy (most estable) configuration posible. In the example belowe we will used a simple classical mechanics simulation to study the formation of a water dimer.

In [ ]:
from ase.optimize import LBFGS
from ase.calculators.emt import EMT

opt = LBFGS(atoms)
opt.run(fmax=1e-3)

view(atoms)

## Water dimer from quantum mechanics: First principles calculation

Unfortunately, the classical simulation above is unable to reproduce the well-known interactions between water molecules that lead to its peculiar properties. In the boxes below we will used DFT simulations.

In [ ]:
!sbatch qe/H2O_dimer.sh

In [ ]:
!squeue --me

In [ ]:
from ase.calculators.espresso import Espresso
from ase.io.espresso import write_espresso_in
import ase
from ase.io import read, write


atoms_opt= ase.io.read('qe/H2O_dimer.out')


write('cif/H2O_opt.cif', atoms_opt, format='cif')


atoms_opt = read('cif/H2O_opt.cif')

view(atoms_opt)

### Water molecule thermal dissociation

The production of $H_2$ is a key in the production of clean energy. As we show at the beggining of the tutorial, $H_2$ combines with $O_2$ to form $H_2O$ and releases a large amount of energy without producing any harmful product. Unfortunately, $H_2$ is found in nature forming other molecules, being $H_2O$ the most aboundant. In the box below we will simulate the thermal decomposition of $H_{2}O$ into $H_{2}$ and $O_{2}$.

Start the simulation at room Temperature (300K).

Breaking the $H-O$ bonds is energetically very demanding. A way to provide the requried energy can be provided by increasing the temperature. The thermal dissociation of $H_{2}O$ is a partial reaction, that produces many subproducts, the conversion into $H_{2}$ and $O_{2}$ is estimated to be only 2.69% at 2500K.

Try 2000K, 3000K and 4000K.

*Sometimes the water molecules in a molecular dynamics simulation will randomly move away from each other. In this case, repeat the simulation.

In [ ]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md import Langevin

Temperature = 300 #Temperature in Kelvin

atoms_opt = read('cif/H2O_opt.cif')
atoms_opt.calc = EMT()

MaxwellBoltzmannDistribution(atoms_opt, temperature_K=Temperature)
dyn = Langevin(atoms_opt, 0.5 * units.fs, friction=0.1, temperature_K=Temperature)


traj = Trajectory('traj/h2o'+str(Temperature)+'.traj', 'w', atoms_opt)
dyn.attach(traj.write, interval=10)

# Now run the dynamics
dyn.run(3000)

frames = ase.io.read('traj/h2o'+str(Temperature)+'.traj', index=':')
view(frames)

### Quantifying the dissociation energy

The dissociation of a water moleculerequires the breaking of the O-H bonds. Here we will used a technique called Nudge Elastic band to quantify the energy barrier (which is linked to the kinetics of the reaction) to do so.

The first step is to build a box with a single water molecule in it.

In [ ]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import LBFGS
from ase.io import read, write
from ase.visualize import view

atoms = molecule('H2O',vacuum=10)

atoms.calc = EMT()

# Initial state
opt = LBFGS(atoms)
opt.run(fmax=0.01)

# Use EMT potential:
atoms.calc = EMT()

# Initial state
opt = LBFGS(atoms)
opt.run(fmax=0.01)
write('traj/initial.traj', atoms)

view(atoms)

Now, we want to define the final state in which oneof the $H$ in the $H_2O$ molecule dissociates. The $H$ atom is located a a distance far enough so that it does not interact with the $OH$ part. 

In [ ]:
# Final state:
atoms[-1].position[0] = atoms[-1].position[0] + 3

atoms.calc = EMT()

# Initial state
opt = LBFGS(atoms)
opt.run(fmax=0.01)

print('final state:', atoms.get_potential_energy())
write('traj/final.traj', atoms)

# and relax.
view(atoms)

Here we run the Nudged elastic band method.

In [ ]:
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.io import read
from ase.neb import NEB
from ase.optimize import LBFGS
import ase.io

initial = read('traj/initial.traj')
final = read('traj/final.traj')

constraint = FixAtoms(mask=[atom.tag > 1 for atom in initial])

images = [initial]
for i in range(9):
    image = initial.copy()
    image.calc = EMT()
    image.set_constraint(constraint)
    images.append(image)


images.append(final)

neb = NEB(images)
neb.interpolate(apply_constraint=True)
qn = LBFGS(neb, trajectory='traj/neb.traj')
qn.run(fmax=0.1)

frames = ase.io.read('traj/neb.traj', index=':')
view(frames)

We can plot the results as a graph where the X axis represents the distance between the atoms and Y axis represents the energy.

In [ ]:
import matplotlib.pyplot as plt

from ase.io import read
from ase.neb import NEBTools

images = read('traj/neb.traj@-9:')

nebtools = NEBTools(images)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools.get_barrier()

# Get the barrier without any interpolation between highest images.
Ef, dE = nebtools.get_barrier(fit=False)

# Get the actual maximum force at this point in the simulation.
max_force = nebtools.get_fmax()

# Create a figure like that coming from ASE-GUI.
fig = nebtools.plot_band()
fig.show()